# POS Tagging con Spacy

## Instalación de Dependencias (Ejecutar primero)

In [49]:
!pip install -q nltk spacy pandas matplotlib seaborn

# Descargar modelo de Spacy
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "spacy", "download", "en_core_web_sm", "-q"])

print("✅ Todas las dependencias instaladas correctamente")


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ Todas las dependencias instaladas correctamente


## 1. Importar Librerías Necesarias

In [50]:
# Importar todas las librerías necesarias
import spacy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from src.utils import path
from collections import Counter
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("✓ Librerías importadas correctamente")

✓ Librerías importadas correctamente


In [51]:
# Descargar recursos necesarios de Spacy (si no están ya instalados)
import spacy

print("Cargando recursos de Spacy...\n")

# Cargar modelo de Spacy en inglés
print("Cargando modelo de Spacy...")
try:
    nlp = spacy.load("en_core_web_sm")
    print("✓ Modelo de Spacy cargado correctamente")
except OSError:
    print("⚠ Modelo no encontrado. Instalando...")
    import subprocess
    subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"], check=True)
    nlp = spacy.load("en_core_web_sm")
    print("✓ Modelo de Spacy instalado y cargado")

print("\n" + "="*60)
print("¡Listo para comenzar con el POS Tagging!")
print("="*60)

Cargando recursos de Spacy...

Cargando modelo de Spacy...
✓ Modelo de Spacy cargado correctamente

¡Listo para comenzar con el POS Tagging!


## Carga del Corpus

In [52]:
directorio_proyecto = path.obtener_ruta_local()
df = pd.read_csv(directorio_proyecto+'\\data\\processed\\corpus_canciones.csv',delimiter = ',',decimal = ".", encoding='utf-8')
df.head()

,Artist,nombre_cancion,letra_cancion,Periodo,Genero
0,Ariana Grande,"​thank u, next",thought i'd end up with sean but he wasn't a m...,2018.0,pop
1,Ariana Grande,7 rings,yeah breakfast at tiffany's and bottles of bub...,2019.0,pop
2,Ariana Grande,​God is a woman,you you love it how i move you you love it how...,2018.0,pop
3,Ariana Grande,Side To Side,ariana grande nicki minaj i've been here all ...,2016.0,pop
4,Ariana Grande,​​no tears left to cry,right now i'm in a state of mind i wanna be in...,2018.0,pop


## 2. POS Tagging con Spacy

### Tokenización

In [53]:
def realizar_token(letra):
    doc = nlp(letra)
    token = []
    for tok in doc:
        tokens = tok.text
        token.append(tokens)
    return token

tqdm.pandas(desc="Paso 1 Tokenización")
df['tokens'] = df['letra_cancion'].progress_apply(realizar_token)

Paso 1 Tokenización: 100%|██████████| 5205/5205 [06:00<00:00, 14.43it/s]


In [54]:
df

,Artist,nombre_cancion,letra_cancion,Periodo,Genero,tokens
0,Ariana Grande,"​thank u, next",thought i'd end up with sean but he wasn't a m...,2018.0,pop,"[thought, i, 'd, end, up, with, sean, but, he,..."
1,Ariana Grande,7 rings,yeah breakfast at tiffany's and bottles of bub...,2019.0,pop,"[yeah, breakfast, at, tiffany, 's, and, bottle..."
2,Ariana Grande,​God is a woman,you you love it how i move you you love it how...,2018.0,pop,"[you, you, love, it, how, i, move, you, you, l..."
3,Ariana Grande,Side To Side,ariana grande nicki minaj i've been here all ...,2016.0,pop,"[ariana, grande, , nicki, minaj, i, 've, been..."
4,Ariana Grande,​​no tears left to cry,right now i'm in a state of mind i wanna be in...,2018.0,pop,"[right, now, i, 'm, in, a, state, of, mind, i,..."
...,...,...,...,...,...,...
5200,Taylor Swift,Should’ve Said No (Live from Clear Channel Str...,it's strange to think the songs we used to sin...,2008.0,pop,"[it, 's, strange, to, think, the, songs, we, u..."
5201,Taylor Swift,Teardrops on my Guitar (Live from Clear Channe...,drew looks at me i fake a smile so he won't se...,2008.0,pop,"[drew, looks, at, me, i, fake, a, smile, so, h..."
5202,Taylor Swift,Evermore [Forward],to put it plainly we just couldnt stop writing...,2020.0,pop,"[to, put, it, plainly, we, just, could, nt, st..."
5203,Taylor Swift,Tolerate it (Polskie Tłumaczenie),zwrotka siedzę i patrzę jak czytasz z głową p...,2020.0,pop,"[zwrotka, , siedzę, i, patrzę, jak, czytasz, ..."


### Etiquetado POS

In [ ]:
def realizar_etiquetado(tokens_lista):
    """
    Recibe una lista de tokens (strings) y devuelve tuplas (token, tag)
    """
    # Reconstruir el texto y procesarlo con spaCy
    texto = " ".join(tokens_lista)
    doc = nlp(texto)
    etiquetas = [(token.text, token.pos_) for token in doc]
    return etiquetas

tqdm.pandas(desc="Paso 2: Etiquetado POS")
df['Etiquetado_POS'] = df['tokens'].progress_apply(realizar_etiquetado)

Paso 2: Etiquetado POS:  60%|██████    | 3141/5205 [04:57<02:26, 14.13it/s] 

In [ ]:
df

### Borrado de StopWords y NER

In [ ]:
def eliminar_stopwords(etiquetas_pos):
    """
    Recibe una lista de tuplas (token, tag) y elimina las stopwords
    """
    sin_stopwords = [(token, tag) for token, tag in etiquetas_pos
                     if token.lower() not in nlp.Defaults.stop_words]
    return sin_stopwords

tqdm.pandas(desc="Paso 3: Eliminar Stopwords")
df['StopWords'] = df['Etiquetado_POS'].progress_apply(eliminar_stopwords)

In [ ]:
df

### Mayúsculas y minúsculas

In [ ]:
def aplicar_minusculas(etiquetas_pos):
    """
    Recibe lista de tuplas (token, tag) y convierte tokens a minúsculas
    """
    minusculas = [(token.lower(), tag) for token, tag in etiquetas_pos]
    return minusculas

tqdm.pandas(desc="Paso 4: Aplicar Minúsculas")
df['Minusculas'] = df['StopWords'].progress_apply(aplicar_minusculas)

In [ ]:
df

### Lematización

In [ ]:
def aplicar_lematizacion(tuplas_tokens):
    """
    Recibe lista de tuplas (token, tag) y devuelve (lemma, tag)
    """
    # Reconstruir el texto desde las tuplas
    texto = " ".join([token for token, tag in tuplas_tokens])
    doc = nlp(texto)

    # Obtener lemmas con sus tags
    lemas = [(token.lemma_, token.tag_) for token in doc]
    return lemas

tqdm.pandas(desc="Paso 5: Lematización")
df['Lematizado'] = df['Minusculas'].progress_apply(aplicar_lematizacion)

In [ ]:
df

In [ ]:
# ============================================================================
# VISUALIZACIÓN DE DISTRIBUCIÓN DE POS TAGS
# ============================================================================

print("\n" + "="*80)
print("VISUALIZACIÓN DE DISTRIBUCIÓN DE POS TAGS")
print("="*80 + "\n")

# Recolectar TODOS los POS tags de todas las canciones
def extraer_todos_pos_tags(df, columna_pos='Lematizado'):
   todos_tags = []
   for pos_tags_list in df[columna_pos]:
        for tupla in pos_tags_list:
            tag = tupla[1]  # El segundo elemento es el tag
            todos_tags.append(tag)
        return todos_tags

# Extraer todos los tags
print("Extrayendo POS tags de todas las canciones...")
todos_pos_tags = extraer_todos_pos_tags(df, 'Lematizado')

# Contar frecuencias
pos_counts = Counter(todos_pos_tags)

# Ordenar por frecuencia (mayor a menor)
pos_counts_sorted = dict(sorted(pos_counts.items(), key=lambda x: x[1], reverse=True))

print(f"✓ Total de palabras analizadas: {len(todos_pos_tags):,}")
print(f"✓ Tipos de POS tags encontrados: {len(pos_counts)}")

# Visualización
fig, ax = plt.subplots(figsize=(14, 7))

pos_names = list(pos_counts_sorted.keys())
pos_values = list(pos_counts_sorted.values())

colors = plt.cm.Set3(range(len(pos_names)))
bars = ax.bar(pos_names, pos_values, color=colors, edgecolor='black', linewidth=1.5)

# Agregar valores en las barras
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height):,}',
            ha='center', va='bottom', fontweight='bold', fontsize=9)

ax.set_xlabel('Categoría Gramatical (POS)', fontsize=12, fontweight='bold')
ax.set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
ax.set_title(f'Distribución de Etiquetas POS en {len(df):,} Canciones',
             fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Rotar etiquetas si hay muchas
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

# Mostrar estadísticas
print("\n" + "="*80)
print("ESTADÍSTICAS DE POS TAGS")
print("="*80)
print(f"\n Total de palabras (después de normalización): {len(todos_pos_tags):,}")
print(f" Tipos diferentes de POS tags: {len(pos_counts)}")

print("\n TOP 10 POS TAGS MÁS FRECUENTES:")
for i, (tag, count) in enumerate(list(pos_counts_sorted.items())[:10], 1):
    porcentaje = (count / len(todos_pos_tags)) * 100
    print(f"{i:2}. {tag:5} → {count:8,} ocurrencias ({porcentaje:5.2f}%)")

### Guardar Corpus

In [ ]:
df.to_csv(directorio_proyecto+'\\data\\results\\corpus_canciones_spicy.csv', index=False)